In [ ]:
import os
import time
import PIL
import pandas as pd
import keras
import keras.applications
from keras.layers import Conv2D, Dense, Flatten, BatchNormalization, Dropout, MaxPooling2D
from keras.applications.mobilenet import preprocess_input
from keras.models import Sequential
from keras import optimizers
import keras.layers as layers
from keras.callbacks import ModelCheckpoint
from google.colab import drive
import skimage.transform
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
%matplotlib inline
!git clone -l -s https://github.com/Dhruv-Sabharwal/GAN_Face_Creator data
%cd data
!ls
drive.mount('/content/drive')

In [ ]:
# We will train the CNN to classify only one attribute at once. Hence, we will have n CNN classifying models for n attributes.
df=pd.read_csv('celeba-dataset/list_attr_celeba.csv') 

# We will use equal number of '+1' and '-1' images of the attribute we want to classify
labels1=df.loc[df['Smiling']==1]
labels1=labels1[:10000]
labels2=df.loc[df['Smiling']==-1]
labels2=labels2[:10000]
data=pd.concat([labels1,labels2])

# We are going to drop all columns except the attribute we want to classify
drop_column=['5_o_Clock_Shadow','Arched_Eyebrows','Attractive','Bags_Under_Eyes','Bald','Bangs','Big_Lips','Big_Nose','High_Cheekbones','Black_Hair','Blurry','Narrow_Eyes','Bushy_Eyebrows','No_Beard','Double_Chin','Eyeglasses','Goatee','Gray_Hair','Heavy_Makeup','Receding_Hairline','Blond_Hair','Pale_Skin','Chubby','Oval_Face','Brown_Hair','Mustache','Pointy_Nose','Young','Rosy_Cheeks','Sideburns','Straight_Hair','Mouth_Slightly_Open','Wavy_Hair','Wearing_Earrings','Wearing_Hat','Wearing_Lipstick','Wearing_Necklace','Wearing_Necktie','Male']
data= data.drop(drop_column,axis=1,inplace=False)

# Shuffling data
data = data.sample(frac=1)

In [ ]:
labels =[] 
# Iterate over each row and read 'image_id' and corresponding attribute value into a list
for index, rows in data.iterrows():
  my_list =[rows.image_id, rows.Smiling] 
  labels.append(my_list) 

# Read images corresponding to 'image_id' into images
import imageio
def data_loader(data_dir):
  images=[]
  i_names=[]
  for i in range(20000):
    i_names.append(labels[i][0])
    images.append(imageio.imread(data_dir+'/'+labels[i][0]))
  return i_names,images
data_directory="celeba-dataset/img_align_celeba/img_align_celeba"
i_names,images=data_loader(data_directory)

#Deleting first column ('image_id')
for x in labels:
  del x[0]

# Reshaping the images to preferred size
reshaped_images=[]
for i in images:
  reshaped_images.append(skimage.transform.resize(i,(80,80), mode ='constant'))
reshaped_images= np.array(reshaped_images)
labels=np.asarray(labels)

# Defining the CNN model

In [ ]:
# Defining the CNN model
filepath="/content/drive/My Drive/AML-Proj-Attributes/Smiling_Best_Weights.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

size_output = 1
model = Sequential() #The Sequential model is a linear stack of layers.
# Conv2D - Applies 32 convolution filters of size 3x3 each.
model.add(Conv2D(32, (9, 9), padding='same', activation='relu', input_shape=(80,80,3)))  #When using this layer as the first layer in a model, we provide the keyword argument "input_shape"
model.add(BatchNormalization())
model.add(Dropout(0.25)) #To help prevent overfitting
 
model.add(Conv2D(64, (7, 7), strides=(2, 2), padding='same', activation='relu')) # Applies 64 convolution filters of size 3x3 each.
model.add(BatchNormalization())
model.add(Dropout(0.25)) #To help prevent overfitting
 
model.add(Conv2D(64, (5, 5), strides=(2, 2), padding='same', activation='relu')) # Applies 128 convolution filters of size 3x3 each.
model.add(BatchNormalization())
model.add(Dropout(0.25)) #To help prevent overfitting

model.add(Conv2D(64, (3, 3), strides=(2, 2), padding='same', activation='relu')) # Applies 128 convolution filters of size 3x3 each.
model.add(BatchNormalization())
model.add(Dropout(0.25)) #To help prevent overfitting

model.add(Conv2D(64, (3, 3), strides=(2, 2), padding='same', activation='relu')) # Applies 128 convolution filters of size 3x3 each.
model.add(BatchNormalization())
model.add(Dropout(0.25)) #To help prevent overfitting

# Classifier 
model.add(Flatten())
model.add(Dense(512, activation='sigmoid'))
model.add(Dropout(0.5))                 
#model.add(Dense(512, activation='sigmoid'))
#model.add(Dropout(0.5))
model.add(Dense(256, activation='sigmoid'))
model.add(Dropout(0.5))
#model.add(Dense(256, activation='sigmoid'))
#model.add(Dropout(0.5))
model.add(Dense(size_output, activation='tanh'))

model.compile(optimizer='sgd', loss='mean_squared_error',metrics=['accuracy'])

In [ ]:
history=model.fit(x=reshaped_images, y=labels, batch_size=128, epochs=50, verbose=1, callbacks=callbacks_list, validation_split=0.15, shuffle=True, initial_epoch=0)

In [ ]:
model.load_weights("/content/drive/My Drive/AML-Proj-Attributes/Smiling_Best_Weights.hdf5")

In [ ]:
model.save("/content/drive/My Drive/AML-Proj-Attributes/Smiling_Model.h5")

# Inference

In [ ]:
data_dir="celeba-dataset/img_align_celeba/img_align_celeba"
image_test=imageio.imread(data_dir + '/' + '062737.jpg')
image_test = skimage.transform.resize(image_test,(80,80,3), mode ='constant')
image_test = np.expand_dims(image_test, axis=0)
plt.imshow(image_test[0])
prediction=model.predict(image_test)
print(prediction)